In [7]:
import pyupbit as up
import datetime 
import numpy as np 
import pandas as pd 
import time 
import random 
import os 
import math  
import matplotlib.pyplot as plt

In [8]:
def simplex_proj(y):
    """ Projection of y onto simplex. """
    m = len(y)
    bget = False

    s = sorted(y, reverse=True)
    tmpsum = 0.

    for ii in range(m-1):
        tmpsum = tmpsum + s[ii]
        tmax = (tmpsum - 1) / (ii + 1);
        if tmax >= s[ii+1]:
            bget = True
            break

    if not bget:
        tmax = (tmpsum + s[m-1] -1)/m
        
    y = np.asarray(y)

    return np.maximum(y-tmax,0.)



# Demo Run 

Check if the model runs correctly on dummy data

In [9]:
import numpy as np
import pandas as pd
# from utils import *

class OLMAR:
    def __init__(self, epsilon, window_size):
        self.epsilon = epsilon
        self.window_size = window_size
        self.cumulative_wealth = 1.0

    def olmar(self, current_portfolio, predicted_returns, window_size):
        avg_x = np.sum([x*y for x,y in zip([1 for i in predicted_returns], predicted_returns)])
        avg_x *= 1.0 / float(len(predicted_returns))
        # numerator
        temp_right = self.epsilon - sum([x*y for x,y in zip(current_portfolio, predicted_returns)])
        # denominator
        temp_bottom1 = np.array(predicted_returns)
        temp_bottom2 = np.array([avg_x for i in predicted_returns])
        dist = np.sum(np.power((temp_bottom1 - temp_bottom2), 2))
        lambda_1 = max(0.0, temp_right/dist)
        # update portfolio
        np_current_portfolio = np.array(current_portfolio)
        new_portfolio = np_current_portfolio + lambda_1 * (temp_bottom1 - temp_bottom2)
        new_portfolio = simplex_proj(new_portfolio.tolist())
        return new_portfolio

    def predict(self, current_portfolio, current_return, previous_prices):
        # calculate cumulative wealth
        self.cumulative_wealth *= np.sum([x*y for x,y in zip(current_portfolio, current_return)])
        m = len(previous_prices[0]) # number of assets
        predicted_returns = []
        for i in range(m): # for each asset, calculate predicted return
            temp = 0
            for j in range(self.window_size):
                temp += previous_prices[j][i]
            predicted_return = (1/previous_prices[0][i]) * (1/self.window_size) * temp
            predicted_returns.append(predicted_return)

        new_portfolio = self.olmar(current_portfolio, predicted_returns, self.window_size)
        return new_portfolio

    def get_cumulative_wealth(self):
        return self.cumulative_wealth

olmar = OLMAR(10, 5) # epsilon = 10, w = 5
'''
current time step is t
since our window size is w = 5, we get the previous timesteps t,t-1,t-2,t-3,t-4
let us assume we have 4 assets (m = 4)
we repeat the below process for n (indefinite number of) timesteps
'''
current_portfolio = [0.25, 0.25, 0.25, 0.25]
previous_prices = [[100,120,130,110],  # t
                   [110,121,90,80], # t-1
                   [120,140,85,75], # t-2
                   [140,200,75,51], # t-3
                   [300,240,80,80]] # t-4
current_returns = [100/110, 120/121, 130/90, 110/80]


new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)

print("portfolio at time t = {}".format(current_portfolio))
print("predicted portfolio at time t+1 = {}".format(new_portfolio))
print("cumulative wealth = {}".format(olmar.get_cumulative_wealth()))


portfolio at time t = [0.25, 0.25, 0.25, 0.25]
predicted portfolio at time t+1 = [1. 0. 0. 0.]
cumulative wealth = 1.180067722681359


# Get Price Information

Get prices from 2020/12/29 to 2021/06/15 for every 4 hour periods

In [88]:
def get_dataframes(coin_name, interval): 
    january = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210131")  
    february = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210228") 
    march = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210331") 
    april = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210430") 
    may = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210531") 
    june = up.get_ohlcv("KRW-"+str(coin_name),interval=interval,to="20210616") 
    df = pd.concat([january, february, march, april, may, june]) 
    df.drop_duplicates(keep=False,inplace=True) 
    return df['close'].values

In [89]:
btc = get_dataframes("BTC", "minute60") 

In [58]:
eth = get_dataframes("ETH", "minute60") 

In [59]:
ada = get_dataframes("ADA", "minute60") 

In [60]:
xrp = get_dataframes("XRP", "minute60") 

In [61]:
dot = get_dataframes("DOT", "minute60") 

In [62]:
price_data = [btc, eth, ada, xrp, dot] 

In [63]:
price_data = np.asarray(price_data)

In [64]:
price_data.shape

(5, 1200)

# Backtest without commissions

In [65]:
epsilon = 10 
m = 5
w = 5
olmar = OLMAR(epsilon, w)

cnt = 1 
current_portfolio = [1/m for i in range(m)] 
for i in range(w-1,len(btc)):
    current_returns = [] 
    for j in range(m): 
        ret = price_data[j][i]/price_data[j][i-1] 
        current_returns.append(ret) 
    current_returns = np.asarray(current_returns) 
    
    previous_prices = [] 
    sub = 0 
    for j in range(w): 
        previous_price = [] 
        for k in range(m):
            previous_price.append(price_data[k][i-j])
        previous_prices.append(previous_price)  
        
    previous_prices = np.asarray(previous_prices)  
    
    
    new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)  
    print("Iteration = {}".format(cnt))
    print("portfolio at time t = {}".format(current_portfolio))
    print("predicted portfolio at time t+1 = {}".format(new_portfolio))  
    
    print("cumulative wealth = {}".format(olmar.get_cumulative_wealth()))
    print()
    cnt += 1 
    current_portfolio = new_portfolio 



Iteration = 1
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 0.9995512618489771

Iteration = 2
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 0.9973822536858329

Iteration = 3
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 0.9819189629310138

Iteration = 4
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 0.9913705893121678

Iteration = 5
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0.08659165 0.         0.         0.         0.91340835]
cumulative wealth = 0.9934709507302022

Iteration = 6
portfolio at time t = [0.08659165 0.         0.         0.         0.91340835]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.0023503064998809

Iteration = 7
portfolio at time t = [1. 0. 0. 0

portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 1.3047709992167105

Iteration = 162
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.3260907867856109

Iteration = 163
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 1.3124548918058103

Iteration = 164
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 1.320831325445149

Iteration = 165
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.3173826796868064

Iteration = 166
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.295951765855899

Iteration = 167
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.28859638201377

Iteration = 329
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.0122272394929572

Iteration = 330
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.0912287848548865

Iteration = 331
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.1007599201713516

Iteration = 332
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.1031311360164993

Iteration = 333
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.0707245194661485

Iteration = 334
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.0242863414780023

Iteration = 335
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth 

portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.616723968228405

Iteration = 491
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.616723968228405

Iteration = 492
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.6497032141393024

Iteration = 493
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.699172083005648

Iteration = 494
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.702140554430282

Iteration = 495
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.6769559248083072

Iteration = 496
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.66436360999732



predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.410654275392203

Iteration = 639
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.493607965621561

Iteration = 640
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.549777045959078

Iteration = 641
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.464465342669264

Iteration = 642
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.4783232040399414

Iteration = 643
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.5062615430282142

Iteration = 644
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.534199882016487

Iteration = 645
portfolio at time t = 

Iteration = 807
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 5.768053645587024

Iteration = 808
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 5.848276922994353

Iteration = 809
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 5.798077121080238

Iteration = 810
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 5.798077121080238

Iteration = 811
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 5.744657558660648

Iteration = 812
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 5.535528123018469

Iteration = 813
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 5.20

portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 7.050601853159364

Iteration = 981
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 6.994339656460604

Iteration = 982
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 7.030250563069604

Iteration = 983
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 6.93356735296845

Iteration = 984
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 7.209805096114604

Iteration = 985
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.0795103980950795

Iteration = 986
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 7.146934306648366

I

portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 7.329270234361251

Iteration = 1158
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 7.314415612889095

Iteration = 1159
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 7.377470919896759

Iteration = 1160
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.377470919896759

Iteration = 1161
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.451245629095727

Iteration = 1162
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.525020338294694

Iteration = 1163
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.525020338294

# With commissions

In [85]:
epsilon = 10 
m = 5
w = 5
# create OLMAR instance 
olmar = OLMAR(epsilon, w)

iterations = 1 
current_portfolio = [1/m for i in range(m)] 
for i in range(w-1,len(btc)):
    current_returns = [] 
    for j in range(m): 
        ret = price_data[j][i]/price_data[j][i-1] 
        current_returns.append(ret) 
    current_returns = np.asarray(current_returns) 
    
    previous_prices = [] 
    sub = 0 
    for j in range(w): 
        previous_price = [] 
        for k in range(m):
            previous_price.append(price_data[k][i-j])
        previous_prices.append(previous_price)  
        
    previous_prices = np.asarray(previous_prices)  
    
    
    new_portfolio = olmar.predict(current_portfolio, current_returns, previous_prices)  
    print("Iteration = {}".format(iterations))
    print("portfolio at time t = {}".format(current_portfolio))
    print("predicted portfolio at time t+1 = {}".format(new_portfolio))  
    
    # compare current and predicted portfolio weights to deduce buy and sell counts 
    buy_sell_cnt = 0 
    for i in range(len(current_portfolio)): 
        buy_sell_cnt += (current_portfolio[i] != new_portfolio[i]) 
    
    
    print("cumulative wealth = {}".format(olmar.get_cumulative_wealth() * np.power((1-0.05/100), buy_sell_cnt)))
    print()
    iterations += 1 
    current_portfolio = new_portfolio 



Iteration = 1
portfolio at time t = [0.2, 0.2, 0.2, 0.2, 0.2]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 0.9970548813233828

Iteration = 2
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 0.9963851207777106

Iteration = 3
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 0.9809372894478237

Iteration = 4
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 0.9913705893121678

Iteration = 5
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0.08659165 0.         0.         0.         0.91340835]
cumulative wealth = 0.9924777281472098

Iteration = 6
portfolio at time t = [0.08659165 0.         0.         0.         0.91340835]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.0013482067809578

Iteration = 7
portfolio at time t = [1. 0. 0. 0

Iteration = 184
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.2929715769310117

Iteration = 185
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.3003007731898995

Iteration = 186
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.308359407552759

Iteration = 187
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.3015915501224784

Iteration = 188
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0.80159797 0.         0.         0.         0.19840203]
cumulative wealth = 1.297746161999463

Iteration = 189
portfolio at time t = [0.80159797 0.         0.         0.         0.19840203]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 1.2859587810144402

Iteration = 190
portfolio at time t = [1. 0. 0

portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 2.925933579096477

Iteration = 356
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 2.910675008321448

Iteration = 357
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 2.929238295014075

Iteration = 358
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.05051943575796

Iteration = 359
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.084354222881781

Iteration = 360
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.047415848955053

Iteration = 361
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 3.0689701581516573

I

predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.650296941062286

Iteration = 508
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.59395975404029

Iteration = 509
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.588506097357376

Iteration = 510
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.6048670674061176

Iteration = 511
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 1. 0. 0.]
cumulative wealth = 3.5903666927761884

Iteration = 512
portfolio at time t = [0. 0. 1. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.5777245565340188

Iteration = 513
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.5598767626822747

Iteration = 514
portfolio at time t =

Iteration = 658
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 3.86371916804434

Iteration = 659
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 3.8709229570626587

Iteration = 660
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.837177170295676

Iteration = 661
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.841017227268637

Iteration = 662
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.7841132683461387

Iteration = 663
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 3.599175401848019

Iteration = 664
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 3.6

Iteration = 816
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 5.176101991784948

Iteration = 817
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 4.766786582277724

Iteration = 818
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 4.849962811650182

Iteration = 819
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 4.9480587480826514

Iteration = 820
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 4.791406233132435

Iteration = 821
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 4.370489210571467

Iteration = 822
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 4.6

portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 7.0872377021725095

Iteration = 980
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 7.050601853159364

Iteration = 981
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 6.987347065389058

Iteration = 982
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 7.030250563069604

Iteration = 983
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 6.93356735296845

Iteration = 984
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [1. 0. 0. 0. 0.]
cumulative wealth = 7.202597093469764

Iteration = 985
portfolio at time t = [1. 0. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.072432657574585

I

Iteration = 1147
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.280363660040271

Iteration = 1148
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 1. 0.]
cumulative wealth = 7.280363660040271

Iteration = 1149
portfolio at time t = [0. 0. 0. 1. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 7.309090488334864

Iteration = 1150
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 1. 0. 0. 0.]
cumulative wealth = 7.393370259373318

Iteration = 1151
portfolio at time t = [0. 1. 0. 0. 0.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 7.36003213472127

Iteration = 1152
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth = 7.320471590622521

Iteration = 1153
portfolio at time t = [0. 0. 0. 0. 1.]
predicted portfolio at time t+1 = [0. 0. 0. 0. 1.]
cumulative wealth 

# Just holding and doing nothing 

What if we just use a constant portfolio 

In [99]:
current_portfolio = [0.2,0.2,0.2,0.2,0.2]
current_return = [price_data[i][-1]/price_data[i][0] for i in range(m)]

np.sum([x*y for x,y in zip(current_portfolio, current_return)])

2.577658304420176